<a href="https://colab.research.google.com/github/eunhwa99/DeepLearning/blob/master/Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
myfile=files.upload()
#파일 불러오기

Saving movies_metadata.csv to movies_metadata.csv


In [5]:
import io #io, pandas 둘다 import 해야함
import numpy as np
import pandas as pd
import json

# Load Dataset
meta=pd.read_csv('movies_metadata.csv', dtype=object)
meta.head() #pandas로 로드할 때 데이터 프레임의 처음 5줄 출력 --> 데이터셋이 어떻게 생겼나 관찰

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


JSON: JavaScript Object Notation, 데이터를 저장하거나 전송할 때 많이 사용되는 경량의 DATA 교환 형식
- 사람과 기계 모두 이해하기 쉬우며 용량이 작아서, JSON이 XML을 대체해서 데이터 전송 등에 많이 사용
- 데이터 포맷일 뿐, 단순히 데이터를 표시하는 표현 방법
- {} 형식, ','로 나열 --> 표현 간단
- 가장 많이 사용되는 포맷: Key-Value Pair

Python은 기본적으로 JSON 표준 라이브러리(json) 제공 (import json)
- Python 타입의 Object를 JSON 문자열로 변경 가능(JSON인코딩)
- JSON 문자열을 다시 Python 타입으로 변환 가능 
(JSON디코딩)

In [6]:
meta=meta[['id','original_title','original_language','genres']]
meta=meta.rename(columns={'id':'movieId'}) #id를 movieId라는 이름으로 바꿈(편의를 위해서)
meta=meta[meta['original_language']=='en'] #영어로 된 언어만
meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [42]:
from google.colab import files
myfile=files.upload()
#파일 불러오기
import io

Saving ratings_small.csv to ratings_small (1).csv


In [43]:
ratings=pd.read_csv('ratings_small.csv')
ratings=ratings[['userId','movieId','rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [44]:
ratings.describe() #데이터의 대략적인 개요 본다
#rating이 최소 0.5, 최대 5.0 이라는 것을 알 수 있다.

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


In [45]:
#데이터셋 가공
#pd.to_numeric(): 문자열을 숫자 타입으로 변환한다
meta.movieId=pd.to_numeric(meta.movieId, errors='coerce') 
ratings.movieId=pd.to_numeric(ratings.movieId,errors='coerce')
#meta의 movieId와 rating의 movieId 를 숫자로 바꾸어서 매칭할 것.

In [55]:
#장르가 json 형태 --> 배열 형태로 가공
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres_str:
        genres_list.append(g['name']) #장르의 이름만 가지고 온다

    return genres_list 

#df.apply(): 각 행에 해당 함수를 적용한다.
meta['genres'] = meta['genres'].apply(parse_genres)
meta.head()

AttributeError: ignored

In [22]:
# 하나의 movieId 즉, 한 영화에 대한 사람들의 평가를 나열
# ratings과 meta를 movieId 기준으로 합친다. inner 방식으로 합친다.
data=pd.merge(ratings, meta, on='movieId',how='inner')

data.head()

,userId,movieId,rating,original_title,original_language,genres
0,1,1371,2.5,Rocky III,en,[r]
1,4,1371,4.0,Rocky III,en,[r]
2,7,1371,3.0,Rocky III,en,[r]
3,19,1371,4.0,Rocky III,en,[r]
4,21,1371,3.0,Rocky III,en,[r]


In [25]:
#pivot table
matrix=data.pivot_table(index='userId',columns='original_title',values='rating')
matrix.head(20)

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,15 Minutes,16 Blocks,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,300,40 Days and 40 Nights,42nd Street,48 Hrs.,4: Rise of the Silver Surfer,5 Card Stud,5 Days of War,50 First Dates,54,8 Mile,88 Minutes,8:46,9 Songs,95 Miles to Go,...,Wild Things,Wild Tigers I Have Known,Wild West Comedy Show: 30 Days & 30 Nights - Hollywood to the Heartland,Wild Wild West,Wild at Heart,Wild in the Country,Will Penny,Willy Wonka & the Chocolate Factory,Wish You Were Here,Within the Woods,Without You I'm Nothing,Wizards of Waverly Place: The Movie,Wizards of the Lost Kingdom,Wolves of Wall Street,Working Girl,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Yamakasi - Les samouraïs des temps modernes,Yankee Doodle Dandy,You Are Not I,You Instead,You Only Live Twice,You're a Big Boy Now,"You, Me and Dupree",Young Adam,Young Black Stallion,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [33]:
#상관관계
GENRE_WEIGHT=0.1

def pearsonR(s1,s2):
  s1_c=s1-s1.mean()
  s2_c=s2-s2.mean()
  return np.sum(s1_c*s2_c)/np.sqrt(np.sum(s1_c**2)*np.sum(s2_c**2))

def recommend(input_movie, matrix, n, similar_genre=True):
  #input으로 받은 input_movie의 장르를 input_genre에 저장
  input_genres=meta[meta['original_title']==input_movie]['genres'].iloc(0)[0]

  result=[]
  for title in matrix.columns:
    if title==input_movie: #같은 영화 추천 받을 필요X
      continue
      
      cor=pearsonR(matrix[input_movie],matrix[title])

      if similar_genre and len(input_genres)>0:
        #지금 비교하고 있는 영화의 장르를 저장
        temp_genres=meta[meta['original_title']==title]['genres'].iloc(0)[0]

        #np.isin(): 배열을 비교하여 똑같은 요소가 있으면 True 반환
        same_count=np.sum(np.isin(input_genres,temp_genres))
        #즉, 같은 장르가 많을 수록 장르의 가중치를 곱해서 더해진다
        cor+=(GENRE_WEIGHT*smae_count)

        if np.isnan(cor):
          continue
        else:
            result.append((title,'{:.2f}'.format(cor),temp_genres))
            
      #rating이 높은 순서대로 (reverse: 내림차순)      
  result.sort(key=lambda r:r[1],reverse=True)

      #n개 만큼만 돌려준다 (n개만 추천해준다,)
  return result[:n]




피어슨 상관관계
- 한 사람이 평가한 2개 영화를 비교해서, 두개가 양의 상관관계를 가지면 2개의 영화는 상관이 큰 것이고 음의 상관관계를 가지면 영화는 상관이 작다.
- 분자가 클수록 상관관계가 크다

이것을 모든 사람에 적용해서 2개의 영화에 대한 상관관계를 모두 합하면 2개의 영화가 큰 상관을 가지고 있는지 아닌지 알 수 있다. (값이 클수록 상관관계가 크다)

In [34]:
#Prediction
recommend_result=recommend('The Dark Knight',matrix,10,similar_genre=True)
#similar_genre: 장르는 비슷한 것일수록 좋다고 사용자가 입력하는 것, 비슷한 장르일수록 가중치를 주겠다.
pd.DataFrame(recommend_result, columns=['Title','Correlation','Genre'])

,Title,Correlation,Genre
